In [0]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### PySpark Import & Utilities functions

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *

In [0]:
# Add project root to Python path so shared transformation utilities can be reused
import os
import sys

project_path = os.path.join(os.getcwd(), "..","..")
sys.path.append(project_path)
from utils.lookup_tables import *
from utils.manual_upds_and_dels_westpac import *

#print(os.path.join(os.getcwd(), "..",".."))

## 1) Source ingestion & Transformed (Auto Loader / streaming)

In [0]:
df_westpac = spark.readStream.format("cloudFiles")\
                .option("cloudFiles.format", "parquet")\
                .option("cloudFiles.schemaLocation","abfss://silver@storagebudgetproject.dfs.core.windows.net/westpac/checkpoint")\
                .option("schemaEvolutionMode", "addNewColumns")\
                .load("abfss://bronze@storagebudgetproject.dfs.core.windows.net/westpac")

In [0]:
df_westpac = (
    df_westpac
    .withColumn(
        "Debit_Amount",
        when(trim(col("Debit_Amount")) != "", col("Debit_Amount"))
        .otherwise(None)
    )
    .withColumn(
        "Credit_Amount",
        when(trim(col("Credit_Amount")) != "", col("Credit_Amount"))
        .otherwise(None)
    )
)

In [0]:
# ----------------------------
# 1) base
# ----------------------------
df_base = (
    df_westpac
    .drop("_rescued_data","Bank_Account","Categories","Balance","Serial")
    .where(col("Date").isNotNull())
    .withColumn("Date", to_date(col("Date"), "dd/MM/yyyy"))
    .withColumn("Description", upper(trim(col("Narrative"))))
    .withColumn(
        "AmountText",
        when(col("Debit_Amount").isNull() & col("Credit_Amount").isNotNull(),
             col("Credit_Amount"))
        .when(col("Credit_Amount").isNull() & col("Debit_Amount").isNotNull(),
              concat(lit("-"), col("Debit_Amount")))
        .otherwise(lit("")))
    .withColumn("Amount", col("AmountText").cast("double"))
    .drop("AmountText","Credit_Amount","Debit_Amount","Narrative")  
)

In [0]:
# ----------------------------
# 2) typed
# ----------------------------
df_typed = (
    df_base
    .withColumn(
        "Type",
         when(col("Amount") < 0, lit("Expenses"))
         .when(col("Amount") > 0, lit("Income"))
         .otherwise(lit(None))
    )
)

In [0]:
# ----------------------------
# 3) cleaned (description_lookup)
# ----------------------------

# Apply utility functions - lookup_tables
df_cleaned = description_lookup.apply(df_typed)

In [0]:
# ----------------------------
# 4) categorized (apple.com override else category_lookup else n/a)
# ----------------------------

# Apply utility functions - lookup_tables
df_categorized = category_lookup.apply(df_cleaned)

df_categorized = (
    df_categorized
    .withColumn(
        "category",
        when((col("Description").like("%APPLE.COM%")) & (abs(col("Amount")) >= 5), lit("Entertainment"))
         .when((col("Description").like("%APPLE.COM%")) & (abs(col("Amount")) <  5), lit("Subscription"))
         .when((col("Description").like("%INTEREST PAID%TERM DEPOSIT%")), lit("Term Deposit Interest"))
         .when((col("Description").like("%INTEREST PAID%")), lit("Saving Interest"))
         .when(col("category").isNotNull(), col("category"))
         .otherwise(lit(None))
    )
)




In [0]:
# ----------------------------
# 5) location (location_lookup else n/a)
# ----------------------------

# Apply utility functions - lookup_tables
df_location = location_lookup.apply(df_categorized)

In [0]:
# ----------------------------
# 6) noted (apple/tpg overrides else note_lookup)
# ----------------------------

# Apply utility functions - lookup_tables
df_noted = note_lookup.apply(df_location)

df_noted = (
    df_noted
    .withColumn(
        "note",
        when((col("Description").like("%APPLE.COM%")) & (abs(col("Amount")) >= 5), lit("apple store"))
         .when((col("Description").like("%APPLE.COM%")) & (abs(col("Amount")) <  5), lit("icloud"))
         .when((col("Description").like("%TPG INTERNET%")) & (abs(col("Amount")) >  35), lit("internet/home"))
         .when((col("Description").like("%TPG INTERNET%")) & (abs(col("Amount")) <= 35), lit("internet/phone"))
         .when(col("note").isNotNull(), col("note"))
         .otherwise(lit(None))
    )
)


In [0]:
# ----------------------------
# 7) Apply manual updates & deletes
# ----------------------------

# Apply utility functions - manual_upds_and_dels_westpac
df_silver = westpac_manual_updates(df_noted)

df_silver = westpac_manual_deletes(df_silver)

Date,updated_at,Description,Amount,Type,cleaned_desc,category,location,note
2025-08-29,2026-01-15T05:28:19.000Z,INTEREST PAID,9.93,Income,INTEREST PAID,Saving Interest,null,null
2025-08-25,2026-01-15T05:28:19.000Z,INTEREST PAID ON 0000000 TERM DEPOSIT 131746,849.32,Income,INTEREST PAID ON 0000000 TERM DEPOSIT 131746,Term Deposit Interest,null,null
2025-08-12,2026-01-15T05:28:19.000Z,DEPOSIT ONLINE 2499646 TFR WESTPAC CHO SPARE FUNDS,100.0,Income,DEPOSIT ONLINE 2499646 TFR WESTPAC CHO SPARE FUNDS,Personal,null,null
2025-08-04,2026-01-15T05:28:19.000Z,WITHDRAWAL MOBILE 1326383 TFR WESTPAC CHO SPARE FUNDS,-2500.0,Expenses,WITHDRAWAL MOBILE 1326383 TFR WESTPAC CHO SPARE FUNDS,Personal,null,null
2025-08-04,2026-01-15T05:28:19.000Z,DEPOSIT ONLINE 2384094 TFR WESTPAC CHO SPARE FUNDS,500.0,Income,DEPOSIT ONLINE 2384094 TFR WESTPAC CHO SPARE FUNDS,Personal,null,null
2025-07-31,2026-01-15T05:28:19.000Z,INTEREST PAID (INCLUDES BONUS OF $106.97),117.4,Income,INTEREST PAID (INCLUDES BONUS OF $106.97),Saving Interest,null,null
2025-07-31,2026-01-15T05:28:19.000Z,INTEREST PAID - PROMOTIONAL,12.7,Income,INTEREST PAID - PROMOTIONAL,Saving Interest,null,null
2025-07-23,2026-01-15T05:28:19.000Z,INTEREST PAID ON 0000000 TERM DEPOSIT 131746,934.93,Income,INTEREST PAID ON 0000000 TERM DEPOSIT 131746,Term Deposit Interest,null,null
2025-07-23,2026-01-15T05:28:19.000Z,DEPOSIT ONLINE 2828075 TFR WESTPAC CHO MAX INTEREST BEFOR,500.0,Income,DEPOSIT ONLINE 2828075 TFR WESTPAC CHO MAX INTEREST BEFOR,Personal,null,null
2025-07-23,2026-01-15T05:28:19.000Z,DEPOSIT ONLINE 2827258 TFR WESTPAC CHO SPARE FUNDS,1500.0,Income,DEPOSIT ONLINE 2827258 TFR WESTPAC CHO SPARE FUNDS,Personal,null,null


In [0]:
# ----------------------------
# 8) Bank name and payment method
# ----------------------------
df_silver = (
    df_silver
    .withColumn("bank", lit("westpac"))
    .withColumn("payment",
        when((col("category") == 'Personal'), lit("Personal"))
         .when((col("location") == 'online') & (col("category") != 'Personal'), lit("Online"))
         .when((col("location") != 'online') & (col("category") != 'Personal'), lit("On-site"))
         .otherwise(lit(None))
    )
)

In [0]:
df_silver = df_silver.withColumn(
    "transaction_id",
    sha2(concat_ws("||",
        coalesce(col("bank"), lit("")),
        col("Date").cast("string"),
        col("Amount").cast("string"),
        coalesce(col("Description"), lit(""))
    ), 256)
)

In [0]:
df_silver.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation", "abfss://silver@storagebudgetproject.dfs.core.windows.net/westpac/checkpoint")\
    .trigger(once=True)\
    .option("path", "abfss://silver@storagebudgetproject.dfs.core.windows.net/westpac/data")\
    .toTable("budget_cata.silver.westpac")